In [ ]:
!pip install -U "tensorflow-text==2.11.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
!pip install bert-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
!pip install bert-for-tf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30531 sha256=b5604491234188a5d76fe9a04a595fe3fa9d7bd202677cf58388ab132676522e
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=5273aec1fa08e3245e259c2fd65c01f63243579454e8fe59737c775028e81897
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7910 sha256=6609e7c6cf56c1a0193f82070d9d25b0b95f6c50124fe370631423585088f342
  Sto

In [40]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
from bert import tokenization

In [30]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [19]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string
import re

In [5]:
train=pd.read_csv('twittertrain.csv')
test=pd.read_csv('twittertest.csv')

In [10]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
train=train.drop(columns=['id','keyword','location'])
train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [20]:
feat=train[['text']]
label=train[['target']]

In [26]:
ps=PorterStemmer()
wordnet=WordNetLemmatizer()

In [27]:
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [31]:
def preprocessing(data):
    corpus=[]
    for i in range(len(data)):
        #remove urls
        tweet=re.sub(r'http\S+',' ',data[i])
        
        #remove html tags
        tweet=re.sub(r'<.*?>',' ',tweet)
        tweet=re.sub('@\w+([-.]\w+)*',' ',tweet)
        tweet=re.sub('&\w+([-.]\w+)*',' ',tweet)
        
        #remove digits
        tweet=re.sub(r'\d+',' ',tweet)
        
        #remove hashtags
        tweet=re.sub(r'#\w+',' ',tweet)
        #remove punctuations and numbers
        review=re.sub('[^a-zA-Z]',' ',tweet)
        review=review.lower()
        review=review.split()
        
        #review=expand_contractions(review)
        lemma=WordNetLemmatizer()
        review=[lemma.lemmatize(word) for word in review if word not in all_stopwords]
        review=' '.join(review)
        corpus.append(review)
    return corpus

In [32]:
feat['text']=preprocessing(feat['text'])
feat.head()


<ipython-input-32-9d8637472cde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat['text']=preprocessing(feat['text'])


,text
0,deed reason may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked shelter place notified officer ...
3,people receive evacuation order california
4,got sent photo ruby smoke pours school


In [44]:
BertTokenizer=bert_tokenization.FullTokenizer
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                          tranable=False)
vocabulary_file=bert_layer.resolved_object.vocabulary_file.asset_path.numpy()
to_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=BertTokenizer(vocabulary_file,to_lower_case)

tokenizer.tokenize("don't be so judgmental")

tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

NameError: ignored